## Data Cleaning

Now since we have extracted data from the website, it is not cleaned and ready to be analyzed yet. The reviews section will need to be cleaned for punctuations, spellings and other characters. 

In [1]:
#imports

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

#regex
import re

In [3]:
#create a dataframe from csv file
 
cwd = os.getcwd()

df = pd.read_csv("C://Users//saikr//Desktop//webscrapping//BA_reviews.csv", index_col=0)

In [4]:
df.head()

,reviews
0,"✅ Trip Verified | Four very pleasant, on time..."
1,Not Verified | We travel extensively over 25...
2,✅ Trip Verified | Flight delayed an hour due ...
3,✅ Trip Verified | A very full flight made Pre...
4,✅ Trip Verified | The worst airline I’ve ever ...


We will also create a column which mentions if the user is verified or not. 

In [5]:
df['verified'] = df.reviews.str.contains("Trip Verified")

In [6]:
df['verified']

0       True
1      False
2       True
3       True
4       True
       ...  
995     True
996     True
997     True
998     True
999     True
Name: verified, Length: 1000, dtype: bool

### Cleaning Reviews

We will extract the column of reviews into a separate dataframe and clean it for semantic analysis

In [ ]:
import nltk

nltk.download()


#for lemmatization of words we will use nltk library
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
lemma = WordNetLemmatizer()


reviews_data = df.reviews.str.strip("✅ Trip Verified |")

#create an empty list to collect cleaned data corpus
corpus =[]

#loop through each review, remove punctuations, small case it, join it and add it to corpus
for rev in reviews_data:
    rev = re.sub('[^a-zA-Z]',' ', rev)
    rev = rev.lower()
    rev = rev.split()
    rev = [lemma.lemmatize(word) for word in rev if word not in set(stopwords.words("english"))]
    rev = " ".join(rev)
    corpus.append(rev)

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


In [ ]:
# add the corpus to the original dataframe

df['corpus'] = corpus

In [12]:
df.head()

,reviews,verified,corpus
0,✅ Trip Verified | Flight delayed an hour due ...,True,flight delayed hour due bad weather around jfk...
1,✅ Trip Verified | A very full flight made Pre...,True,full flight made premium economy feel little l...
2,✅ Trip Verified | The worst airline I’ve ever ...,True,worst airline ever flown could check flight op...
3,✅ Trip Verified | I am surprised to be able t...,True,surprised able give favourable review flight e...
4,✅ Trip Verified | Flew British Airways on BA ...,True,flew british airway ba london heathrow amsterd...


### Cleaning/Fromat date

In [13]:
df.dtypes

reviews     object
verified      bool
corpus      object
dtype: object

In [14]:
# convert the date to datetime format

df.date = pd.to_datetime(df.date)

AttributeError: 'DataFrame' object has no attribute 'date'

In [93]:
df.date.head()

0   2022-11-07
1   2022-11-07
2   2022-11-05
3   2022-10-31
4   2022-10-31
Name: date, dtype: datetime64[ns]

### Cleaning ratings with stars

In [100]:
#check for unique values
df.stars.unique()

array(['\n\t\t\t\t\t\t\t\t\t\t\t\t\t5', '1', '5', '6', '9', '3', '2', '8',
       '7', '10', '4', 'None'], dtype=object)

In [102]:
# remove the \t and \n from the ratings
df.stars = df.stars.str.strip("\n\t\t\t\t\t\t\t\t\t\t\t\t\t")

In [105]:
df.stars.value_counts()

1       735
2       382
3       379
8       349
10      306
7       299
9       293
5       259
4       227
6       182
None      5
Name: stars, dtype: int64

There are 5 rows having values "None" in the ratings. We will drop all these 5 rows. 

In [110]:
# drop the rows where the value of ratings is None
df.drop(df[df.stars == "None"].index, axis=0, inplace=True)

In [111]:
#check the unique values again
df.stars.unique()

array(['5', '1', '6', '9', '3', '2', '8', '7', '10', '4'], dtype=object)

## Check for null Values

In [112]:
df.isnull().value_counts()

reviews  stars  date   country  verified  corpus
False    False  False  False    False     False     3411
dtype: int64

In [113]:
df.country.isnull().value_counts()

False    3411
Name: country, dtype: int64

We have two missing values for country. For this we can just remove those two reviews (rows) from the dataframe. 

In [114]:
#drop the rows using index where the country value is null
df.drop(df[df.country.isnull() == True].index, axis=0, inplace=True)

In [115]:
df.shape

(3411, 6)

In [116]:
#resetting the index
df.reset_index(drop=True)

,reviews,stars,date,country,verified,corpus
0,Not Verified | Worst experience ever. Outbound...,5,2022-11-07,Italy,False,verified worst experience ever outbound flight...
1,✅ Trip Verified | Check in was a shambles at ...,1,2022-11-07,Malaysia,True,check shamble bwi counter open full flight bag...
2,✅ Trip Verified | Beyond disgusted with the fa...,5,2022-11-05,United Arab Emirates,True,beyond disgusted fact baggage yet delivered we...
3,✅ Trip Verified | On July 19th 2022 I had subm...,1,2022-10-31,United States,True,july th submitted complaint form regard fact b...
4,✅ Trip Verified | I booked the flight on Oct ...,1,2022-10-31,United States,True,booked flight oct cancel flight day learning g...
...,...,...,...,...,...,...
3406,This was a bmi Regional operated flight on a R...,1,2012-08-29,United Kingdom,False,bmi regional operated flight rj manchester hea...
3407,LHR to HAM. Purser addresses all club passenge...,10,2012-08-28,United Kingdom,False,lhr ham purser address club passenger name boa...
3408,My son who had worked for British Airways urge...,10,2011-10-12,United Kingdom,False,son worked british airway urged fly british ai...
3409,London City-New York JFK via Shannon on A318 b...,8,2011-10-11,United States,False,london city new york jfk via shannon really ni...


*****

Now our data is all cleaned and ready for data visualization and data analysis.

In [117]:
# export the cleaned data

df.to_csv(cwd + "/cleaned-BA-reviews.csv")